# Electricity Consumption Forecasting - Modeling

This notebook trains and compares multiple forecasting models to predict next-day household energy consumption.

## Objectives
- Engineer time-series features (lags, rolling statistics, calendar features)
- Split data chronologically for time-series validation
- Train baseline, linear, tree-based, and gradient boosting models
- Tune hyperparameters and select the best model
- Save the final model and preprocessing pipeline for deployment

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid")

PROJECT_ROOT = Path().resolve().parent
DATA_PATH = PROJECT_ROOT / 'data/household_power_consumption.txt'

## Load and prepare daily energy data
Reuse the preprocessing logic from EDA to create daily aggregates.

In [ ]:
dtype_spec = {
    'Date': 'string',
    'Time': 'string',
    'Global_active_power': 'string',
    'Global_reactive_power': 'string',
    'Voltage': 'string',
    'Global_intensity': 'string',
    'Sub_metering_1': 'string',
    'Sub_metering_2': 'string',
    'Sub_metering_3': 'string'
}

df = pd.read_csv(
    DATA_PATH,
    sep=';',
    dtype=dtype_spec,
    na_values='?',
    parse_dates={'timestamp': ['Date', 'Time']},
    dayfirst=True,
    low_memory=False
)

numeric_cols = [
    'Global_active_power',
    'Global_reactive_power',
    'Voltage',
    'Global_intensity',
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]

df[numeric_cols] = df[numeric_cols].astype(float)
df = df.dropna(subset=['Global_active_power']).reset_index(drop=True)
df = df.sort_values('timestamp').set_index('timestamp')
df = df[~df.index.duplicated(keep='first')]

# Create daily energy target
df['energy_kwh'] = df['Global_active_power'] / 60.0
daily_energy = df['energy_kwh'].resample('D').sum().to_frame(name='kwh')
daily_energy = daily_energy[daily_energy['kwh'] > 0]  # Remove zero days

print(f"Daily records: {len(daily_energy)}")
print(f"Date range: {daily_energy.index.min()} to {daily_energy.index.max()}")

## Feature engineering
Create lag features, rolling statistics, and calendar-based features for forecasting.

In [ ]:
def create_features(df, target_col='kwh'):
    """Create time-series features for forecasting."""
    features = df.copy()
    
    # Lag features (previous days)
    for lag in [1, 2, 3, 7, 14]:
        features[f'lag_{lag}'] = features[target_col].shift(lag)
    
    # Rolling statistics
    for window in [7, 14, 30]:
        features[f'rolling_mean_{window}'] = features[target_col].rolling(window=window, min_periods=1).mean()
        features[f'rolling_std_{window}'] = features[target_col].rolling(window=window, min_periods=1).std()
    
    # Calendar features
    features['day_of_week'] = features.index.dayofweek
    features['month'] = features.index.month
    features['day_of_month'] = features.index.day
    features['is_weekend'] = (features.index.dayofweek >= 5).astype(int)
    
    # Fourier terms for seasonality (weekly)
    features['sin_week'] = np.sin(2 * np.pi * features.index.dayofweek / 7)
    features['cos_week'] = np.cos(2 * np.pi * features.index.dayofweek / 7)
    
    # Fourier terms for monthly seasonality
    features['sin_month'] = np.sin(2 * np.pi * features.index.month / 12)
    features['cos_month'] = np.cos(2 * np.pi * features.index.month / 12)
    
    return features

features_df = create_features(daily_energy)
features_df.head(10)

In [ ]:
# Drop rows with NaN from lag features
features_df = features_df.dropna().copy()
print(f"Features shape: {features_df.shape}")
print(f"\nFeature columns: {features_df.columns.tolist()}")

## Prepare train/validation split
Use chronological split: train on earlier data, validate on later data.

In [ ]:
# Use 80/20 split chronologically
split_idx = int(len(features_df) * 0.8)
train = features_df.iloc[:split_idx].copy()
val = features_df.iloc[split_idx:].copy()

print(f"Train: {len(train)} days ({train.index.min()} to {train.index.max()})")
print(f"Validation: {len(val)} days ({val.index.min()} to {val.index.max()})")

# Separate features and target
feature_cols = [c for c in features_df.columns if c != 'kwh']
X_train = train[feature_cols]
y_train = train['kwh']
X_val = val[feature_cols]
y_val = val['kwh']

## Baseline model: Naive forecast
Use yesterday's value as a simple baseline.

In [ ]:
baseline_pred = X_val['lag_1'].values
baseline_rmse = np.sqrt(mean_squared_error(y_val, baseline_pred))
baseline_mae = mean_absolute_error(y_val, baseline_pred)
baseline_r2 = r2_score(y_val, baseline_pred)

print(f"Baseline (yesterday's value):")
print(f"  RMSE: {baseline_rmse:.2f} kWh")
print(f"  MAE: {baseline_mae:.2f} kWh")
print(f"  R²: {baseline_r2:.3f}")

## Train multiple models
Compare Linear Regression, Ridge, Random Forest, and Gradient Boosting.

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (alpha=1.0)': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
}

try:
    from xgboost import XGBRegressor
    models['XGBoost'] = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs=-1)
except ImportError:
    print("XGBoost not available, skipping...")

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    mae = mean_absolute_error(y_val, pred)
    r2 = r2_score(y_val, pred)
    
    results[name] = {
        'model': model,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'predictions': pred
    }
    
    print(f"  RMSE: {rmse:.2f} kWh")
    print(f"  MAE: {mae:.2f} kWh")
    print(f"  R²: {r2:.3f}")

## Compare model performance

In [ ]:
results_df = pd.DataFrame({
    name: {
        'RMSE': res['rmse'],
        'MAE': res['mae'],
        'R²': res['r2']
    }
    for name, res in results.items()
}).T

results_df = results_df.sort_values('RMSE')
results_df

In [ ]:
plt.figure(figsize=(10, 5))
results_df[['RMSE', 'MAE']].plot(kind='bar', ax=plt.gca())
plt.title('Model Comparison: RMSE and MAE')
plt.ylabel('Error (kWh)')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

## Hyperparameter tuning
Tune the best model using grid search with time-series cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Identify best model
best_model_name = results_df.index[0]
print(f"Best model: {best_model_name}")

# Tune Random Forest if it's the best
if 'Random Forest' in best_model_name or 'Random Forest' in results:
    print("\nTuning Random Forest...")
    rf_param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [8, 10, 12],
        'min_samples_split': [2, 5]
    }
    
    tscv = TimeSeriesSplit(n_splits=3)
    rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf_grid = GridSearchCV(rf_base, rf_param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    rf_grid.fit(X_train, y_train)
    
    print(f"Best params: {rf_grid.best_params_}")
    best_model = rf_grid.best_estimator_
    
    # Evaluate tuned model
    tuned_pred = best_model.predict(X_val)
    tuned_rmse = np.sqrt(mean_squared_error(y_val, tuned_pred))
    tuned_mae = mean_absolute_error(y_val, tuned_pred)
    tuned_r2 = r2_score(y_val, tuned_pred)
    
    print(f"\nTuned Random Forest:")
    print(f"  RMSE: {tuned_rmse:.2f} kWh")
    print(f"  MAE: {tuned_mae:.2f} kWh")
    print(f"  R²: {tuned_r2:.3f}")
    
    # Update best model if tuned version is better
    if tuned_rmse < results['Random Forest']['rmse']:
        results['Random Forest (Tuned)'] = {
            'model': best_model,
            'rmse': tuned_rmse,
            'mae': tuned_mae,
            'r2': tuned_r2,
            'predictions': tuned_pred
        }
        best_model_name = 'Random Forest (Tuned)'
        best_model = best_model
    else:
        best_model = results['Random Forest']['model']
        best_model_name = 'Random Forest'
else:
    # Use best untuned model
    best_model = results[best_model_name]['model']
    print(f"\nUsing {best_model_name} as final model")

## Visualize predictions vs actual

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Time series
axes[0].plot(val.index, y_val.values, label='Actual', alpha=0.7, linewidth=1)
axes[0].plot(val.index, results[best_model_name]['predictions'], label='Predicted', alpha=0.7, linewidth=1)
axes[0].set_title(f'{best_model_name} - Predictions vs Actual')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Energy (kWh)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Scatter plot
axes[1].scatter(y_val.values, results[best_model_name]['predictions'], alpha=0.5)
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Perfect prediction')
axes[1].set_xlabel('Actual (kWh)')
axes[1].set_ylabel('Predicted (kWh)')
axes[1].set_title('Prediction Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Feature importance (if available)

In [ ]:
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=importance_df.head(15), x='importance', y='feature')
    plt.title('Top 15 Feature Importances')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 features:")
    print(importance_df.head(10))

## Save final model and preprocessing info
Save the model and feature list for deployment.

In [ ]:
MODEL_DIR = PROJECT_ROOT / 'models'
MODEL_DIR.mkdir(exist_ok=True)

# Save model
model_path = MODEL_DIR / 'electricity_forecast_model.joblib'
joblib.dump(best_model, model_path)
print(f"Model saved to: {model_path}")

# Save feature columns
feature_info = {
    'feature_columns': feature_cols,
    'target_column': 'kwh',
    'model_name': best_model_name,
    'rmse': results[best_model_name]['rmse'],
    'mae': results[best_model_name]['mae'],
    'r2': results[best_model_name]['r2']
}

import json
feature_path = MODEL_DIR / 'model_features.json'
with open(feature_path, 'w') as f:
    json.dump(feature_info, f, indent=2, default=str)
print(f"Feature info saved to: {feature_path}")

print(f"\nFinal model performance:")
print(f"  RMSE: {feature_info['rmse']:.2f} kWh")
print(f"  MAE: {feature_info['mae']:.2f} kWh")
print(f"  R²: {feature_info['r2']:.3f}")